## SiTS Experimental Parameter File Generation
### Author: Katie Murenbeeld
### Date: 10 July 2020

**S**ignals **i**n **T**he **S**oil. This notebook outlines how to create the experimental soil texture (st) and hydraulic properties (hp) netCDF files for use in model simulations and experimentation. 

There are 4 basic steps covered for creating the experimental files for single point simulations.

1. Create an empty dataset or data series
2. Create data arrays for desired st or hp data
3. Add the data arrays into the dataset/dataseries
4. Write out to a netCDF file

For larger scale cases, the above 4 steps can occur after the dimensions and coordinates from the "source" surfdata file are extracted.

## Generate an experimental file for a single point case

### 1. Create an empty dataset or data series.
Create an empty dataseries and set the global attributes for the file.

First, load the appropriate libraries.

In [1]:
# Load appropriate libraries for using xarray

import numpy as np
import pandas as pd
import xarray as xr

Next, create the empty dataset or data series. "A dataset is a dict-like array for aligned DataArray objects." See xarray documentation http://xarray.pydata.org/en/stable/quick-overview.html#datasets.

In [2]:
my_ds = xr.Dataset() 

Then, add appropriate global attributes for the file,

* title 
* description or long_name
* created_by
* institution 

are a few examples.


In [3]:
my_ds.attrs['title'] = 'brief one sentence description of file'
my_ds.attrs['description'] = 'more detailed description of the file and purpose' # can also be called 'long_name'
my_ds.attrs['created_by'] = 'your name'
my_ds.attrs['institution'] = 'your instituion'

# view your empty dataset
my_ds

<xarray.Dataset>
Dimensions:  ()
Data variables:
    *empty*
Attributes:
    title:        brief one sentence description of file
    description:  more detailed description of the file and purpose
    created_by:   your name
    institution:  your instituion

### 2. Create data arrays for the desired st and hp data

This is a very similar step to creating the dataset, but for the array you will set the

* dims or dimenesions
* coordinates
* local attributes for the data or variable

First, set the data variable. Since this is a single point example, you only need one value. In this example I will create a dataarray for saturated hydraulic conductivity *ksat*. Note that lat and lon values can be found in the domain.lnd file used in the single point case.

In [6]:
ksat = 3.4260413e-06

ksat_xar = xr.DataArray(ksat,
                       dims = ['lat', 'lon'],
                       coords = {'lat': [43.0], 
                                'lon': [243.0]})

Next, set the local attributes for the data. Attributes should include at least:

* _FillValue
* units
* long_name


In [7]:
ksat_xar.attrs['long_name'] = 'saturated hydraulic conductivity'
ksat_xar.attrs['_FillValue'] = -9999.9 # For use in the Community Land Model (CLM) the _FillValue needs to be -9999 and not NaN
ksat_xar.attrs['units'] = 'm s-1'

# View your ksat DataArray
ksat_xar

<xarray.DataArray (lat: 1, lon: 1)>
array([[3.4260413e-06]])
Coordinates:
  * lat      (lat) float64 43.0
  * lon      (lon) float64 243.0
Attributes:
    long_name:   saturated hydraulic conductivity
    _FillValue:  -9999.9
    units:       m s-1

Below is a generic example.

In [9]:
data = 4.55

data_xar = xr.DataArray(data,
                       dims = ['lat', 'lon'],
                       coords = {'lat': [43.0],
                                'lon': [243.0]})

In [10]:
data_xar.attrs['long_name'] = 'generic data'
data_xar.attrs['_FillValue'] = -9999.9 # For use in the Community Land Model (CLM) the _FillValue needs to be -9999 and not NaN
data_xar.attrs['units'] = 'units'

# View your generic data DataArray
data_xar

<xarray.DataArray (lat: 1, lon: 1)>
array([[4.55]])
Coordinates:
  * lat      (lat) float64 43.0
  * lon      (lon) float64 243.0
Attributes:
    long_name:   generic data
    _FillValue:  -9999.9
    units:       units

### 3. Add the DataArrays to the empty DataSet

Combine the two DataArrays into one dataset.

In [11]:
my_ds['ksat'] = ksat_xar
my_ds['data'] = data_xar

# View the new DataSet
my_ds

<xarray.Dataset>
Dimensions:  (lat: 1, lon: 1)
Coordinates:
  * lat      (lat) float64 43.0
  * lon      (lon) float64 243.0
Data variables:
    ksat     (lat, lon) float64 3.426e-06
    data     (lat, lon) float64 4.55
Attributes:
    title:        brief one sentence description of file
    description:  more detailed description of the file and purpose
    created_by:   your name
    institution:  your instituion

### 4. Write to a netCDF 

In [65]:
my_ds.to_netcdf('example_sits_spt_exp_file.nc')

And, voila! That is all there is to it for creating the experimental netCDF file for a single point case. 


## Generate an experimental for a regional to global scale case.

For a larger scale case (i.e. larger than a single point 1x1 gridcell case), you may need to use the "source" surfdata file for the case you will create and run. For example, surfdata_360x720cru_78pfts_simyr2000_c170428.nc is a 360x720 gridcell surface data file. One can extract the correct dimensions and coordinates from the "source" to create the experimental netCDF file.

### 1. Create an empty dataset or data series.

In [58]:
my_ds2 = xr.Dataset() 

my_ds2.attrs['title'] = 'brief one sentence description of file'
my_ds2.attrs['description'] = 'more detailed description of the file and purpose' # can also be called 'long_name'
my_ds2.attrs['created_by'] = 'your name'
my_ds2.attrs['institution'] = 'your instituion'

# view your empty dataset
my_ds2

<xarray.Dataset>
Dimensions:  ()
Data variables:
    *empty*
Attributes:
    title:        brief one sentence description of file
    description:  more detailed description of the file and purpose
    created_by:   your name
    institution:  your instituion

Read in the "source" surfdata netCDF that you will overwrite and use in your simulation.

In [13]:
src = xr.open_dataset('surfdata_360x720cru_78pfts_simyr2000_c170428.nc')
src

<xarray.Dataset>
Dimensions:                  (cft: 64, lsmlat: 360, lsmlon: 720, lsmpft: 79, natpft: 15, nglcec: 10, nglcecp1: 11, nlevsoi: 10, nlevurb: 5, numrad: 2, numurbl: 3, time: 12)
Coordinates:
  * natpft                   (natpft) int32 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14
  * cft                      (cft) int32 15 16 17 18 19 20 ... 73 74 75 76 77 78
  * time                     (time) int32 1 2 3 4 5 6 7 8 9 10 11 12
Dimensions without coordinates: lsmlat, lsmlon, lsmpft, nglcec, nglcecp1, nlevsoi, nlevurb, numrad, numurbl
Data variables:
    mxsoil_color             int32 ...
    SOIL_COLOR               (lsmlat, lsmlon) int32 ...
    PCT_SAND                 (nlevsoi, lsmlat, lsmlon) float64 ...
    PCT_CLAY                 (nlevsoi, lsmlat, lsmlon) float64 ...
    ORGANIC                  (nlevsoi, lsmlat, lsmlon) float64 ...
    FMAX                     (lsmlat, lsmlon) float64 ...
    LANDFRAC_PFT             (lsmlat, lsmlon) float64 ...
    PFTDATA_MASK             (lsmlat, lsmlon) int32 ...
    PCT_NATVEG               (lsmlat, lsmlon) float64 ...
    PCT_CROP                 (lsmlat, lsmlon) float64 ...
    PCT_NAT_PFT              (natpft, lsmlat, lsmlon) float64 ...
    PCT_CFT                  (cft, lsmlat, lsmlon) float64 ...
    MONTHLY_LAI              (time, lsmpft, lsmlat, lsmlon) float64 ...
    MONTHLY_SAI              (time, lsmpft, lsmlat, lsmlon) float64 ...
    MONTHLY_HEIGHT_TOP       (time, lsmpft, lsmlat, lsmlon) float64 ...
    MONTHLY_HEIGHT_BOT       (time, lsmpft, lsmlat, lsmlon) float64 ...
    AREA                     (lsmlat, lsmlon) float64 ...
    LONGXY                   (lsmlat, lsmlon) float64 ...
    LATIXY                   (lsmlat, lsmlon) float64 ...
    EF1_BTR                  (lsmlat, lsmlon) float64 ...
    EF1_FET                  (lsmlat, lsmlon) float64 ...
    EF1_FDT                  (lsmlat, lsmlon) float64 ...
    EF1_SHR                  (lsmlat, lsmlon) float64 ...
    EF1_GRS                  (lsmlat, lsmlon) float64 ...
    EF1_CRP                  (lsmlat, lsmlon) float64 ...
    CANYON_HWR               (numurbl, lsmlat, lsmlon) float64 ...
    EM_IMPROAD               (numurbl, lsmlat, lsmlon) float64 ...
    EM_PERROAD               (numurbl, lsmlat, lsmlon) float64 ...
    EM_ROOF                  (numurbl, lsmlat, lsmlon) float64 ...
    EM_WALL                  (numurbl, lsmlat, lsmlon) float64 ...
    HT_ROOF                  (numurbl, lsmlat, lsmlon) float64 ...
    THICK_ROOF               (numurbl, lsmlat, lsmlon) float64 ...
    THICK_WALL               (numurbl, lsmlat, lsmlon) float64 ...
    T_BUILDING_MIN           (numurbl, lsmlat, lsmlon) float64 ...
    WIND_HGT_CANYON          (numurbl, lsmlat, lsmlon) float64 ...
    WTLUNIT_ROOF             (numurbl, lsmlat, lsmlon) float64 ...
    WTROAD_PERV              (numurbl, lsmlat, lsmlon) float64 ...
    ALB_IMPROAD_DIR          (numrad, numurbl, lsmlat, lsmlon) float64 ...
    ALB_IMPROAD_DIF          (numrad, numurbl, lsmlat, lsmlon) float64 ...
    ALB_PERROAD_DIR          (numrad, numurbl, lsmlat, lsmlon) float64 ...
    ALB_PERROAD_DIF          (numrad, numurbl, lsmlat, lsmlon) float64 ...
    ALB_ROOF_DIR             (numrad, numurbl, lsmlat, lsmlon) float64 ...
    ALB_ROOF_DIF             (numrad, numurbl, lsmlat, lsmlon) float64 ...
    ALB_WALL_DIR             (numrad, numurbl, lsmlat, lsmlon) float64 ...
    ALB_WALL_DIF             (numrad, numurbl, lsmlat, lsmlon) float64 ...
    TK_ROOF                  (nlevurb, numurbl, lsmlat, lsmlon) float64 ...
    TK_WALL                  (nlevurb, numurbl, lsmlat, lsmlon) float64 ...
    TK_IMPROAD               (nlevurb, numurbl, lsmlat, lsmlon) float64 ...
    CV_ROOF                  (nlevurb, numurbl, lsmlat, lsmlon) float64 ...
    CV_WALL                  (nlevurb, numurbl, lsmlat, lsmlon) float64 ...
    CV_IMPROAD               (nlevurb, numurbl, lsmlat, lsmlon) float64 ...
    NLEV_IMPROAD             (numurbl, lsmlat, lsmlon) int32 

Notice the list of dimensions and coordinates. 

In [48]:
# Create arrays for the latitutde and longitude.
src_lat = src['lsmlat'].values
src_lon = src['lsmlon'].values

# Create arrays for the coordinate values of latitude and longitude.
src_latixy = src['LATIXY'].values
src_longxy = src['LONGXY'].values

In [41]:
src_lat

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [45]:
src_latixy.shape

(360, 720)

In [49]:
src_latixy[:,0].shape

(360,)

In [50]:
src_longxy.shape

(360, 720)

In [51]:
src_longxy[0,:].shape

(720,)

### 2. Create data arrays for the desired st and hp data

In this example, I will generate two random datasets with the same size as the "source" surfdata file. In real life you will have an array of data already or you will fill in the array with the appropriate or desired values for the st or hp variable.

In [60]:
rn_data = np.random.rand(360,720) # Create a random dataset the same size as the "source" surfdata file.

In [61]:
# Check that the size is correct.
rn_data.shape

(360, 720)

Create the DataArray. Notice that the lat and lon coordinates are set to an index of the LATIXY and LONGXY from the "source" surfdata file.

In [54]:
rn_data_xar = xr.DataArray(rn_data,
                          dims = ['lat', 'lon'],
                          coords = {'lat': src_latixy[:,0],
                                   'lon': src_longxy[0,:]})

rn_data_xar.attrs['long_name'] = 'randomly generated data'
rn_data_xar.attrs['_FillValue'] = -9999.9 # For use in the Community Land Model (CLM) the _FillValue needs to be -9999 and not NaN
rn_data_xar.attrs['units'] = 'units'

# View your generic data DataArray
rn_data_xar

<xarray.DataArray (lat: 360, lon: 720)>
array([[0.44223154, 0.02806406, 0.53306949, ..., 0.83975712, 0.94468427,
        0.14386218],
       [0.09846764, 0.2154158 , 0.66516881, ..., 0.39057398, 0.61956492,
        0.7911828 ],
       [0.27970587, 0.3699499 , 0.9392084 , ..., 0.9858284 , 0.25484082,
        0.67388434],
       ...,
       [0.53380702, 0.82287021, 0.79455553, ..., 0.35252623, 0.61236133,
        0.78523033],
       [0.75040138, 0.56477483, 0.83649264, ..., 0.50296712, 0.60681707,
        0.52972546],
       [0.24160781, 0.324167  , 0.34531791, ..., 0.89379415, 0.8433647 ,
        0.96193871]])
Coordinates:
  * lat      (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon      (lon) float64 0.25 0.75 1.25 1.75 2.25 ... 358.2 358.8 359.2 359.8
Attributes:
    long_name:   randomly generated data
    _FillValue:  -9999.9
    units:       units

Create a second randomly generated DataArray.

In [62]:
rn_data2 = np.random.rand(360,720) # Create a second random dataset the same size as the "source" surfdata file.

rn_data2_xar = xr.DataArray(rn_data2,
                          dims = ['lat', 'lon'],
                          coords = {'lat': src_latixy[:,0],
                                   'lon': src_longxy[0,:]})

rn_data2_xar.attrs['long_name'] = 'randomly generated data second dataset'
rn_data2_xar.attrs['_FillValue'] = -9999.9 # For use in the Community Land Model (CLM) the _FillValue needs to be -9999 and not NaN
rn_data2_xar.attrs['units'] = 'units'

# View your generic data DataArray
rn_data2_xar

<xarray.DataArray (lat: 360, lon: 720)>
array([[0.48032177, 0.0424844 , 0.33991844, ..., 0.56042887, 0.15968536,
        0.17340744],
       [0.28150593, 0.55163269, 0.00779137, ..., 0.1083941 , 0.02041441,
        0.86817927],
       [0.7282668 , 0.6354033 , 0.87376577, ..., 0.38802796, 0.07195205,
        0.50154446],
       ...,
       [0.55783082, 0.18540611, 0.60175603, ..., 0.53384974, 0.8122818 ,
        0.15682836],
       [0.89044058, 0.99496521, 0.97606487, ..., 0.76016445, 0.7398849 ,
        0.95168606],
       [0.42884369, 0.74780905, 0.474116  , ..., 0.98653065, 0.68516274,
        0.78913455]])
Coordinates:
  * lat      (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon      (lon) float64 0.25 0.75 1.25 1.75 2.25 ... 358.2 358.8 359.2 359.8
Attributes:
    long_name:   randomly generated data second dataset
    _FillValue:  -9999.9
    units:       units

### 3. Add the DataArrays to the empty DataSet

Combine the two DataArrays into one dataset.

In [59]:
my_ds2['data1'] = rn_data_xar
my_ds2['data2'] = rn_data2_xar

# View the new DataSet
my_ds2

<xarray.Dataset>
Dimensions:  (lat: 360, lon: 720)
Coordinates:
  * lat      (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon      (lon) float64 0.25 0.75 1.25 1.75 2.25 ... 358.2 358.8 359.2 359.8
Data variables:
    data1    (lat, lon) float64 0.4422 0.02806 0.5331 ... 0.8938 0.8434 0.9619
    data2    (lat, lon) float64 0.4783 0.215 0.8683 0.6664 ... 0.3646 0.0589 0.9
Attributes:
    title:        brief one sentence description of file
    description:  more detailed description of the file and purpose
    created_by:   your name
    institution:  your instituion

### 4. Write to a netCDF 

In [63]:
my_ds2.to_netcdf('example_sits_global_exp_file.nc')